In [28]:
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import seaborn as sns


nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
data = pd.read_csv("depression_detection.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
pattern_url = re.compile(r'\[(.+)\]\(.+\)')  # markdown links
pattern_subreddit = re.compile(r'r/\w+\b')  # subreddits
pattern_user = re.compile(r'u/\w+\b')  # users

# from: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
pattern_emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

def replace_words(string):
    string = pattern_url.sub('_URL_', string)
    string = pattern_subreddit.sub('_SUB_', string)
    string = pattern_user.sub('_USER_', string)
    string = pattern_emoji.sub('', string)
    return string

In [30]:
# remove usernames, links, subs, and emojis
data['text'] = data['text'].apply(replace_words)

# remove reddit anomalies
data = data[data['text'].str.find('[removed]') == -1]
data = data[data['text'].str.find('[deleted]') == -1]

In [34]:
all_words_suicide = []
data['text']=data['text'].apply(lambda w: w.lower() for w in word_tokenize(w))

In [37]:
stopwords_filtered = stopwords.words('english')[:stopwords.words('english').index('aren')]

In [38]:
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_filtered)]))

In [42]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['class'] = le.fit_transform(data['class'])

In [56]:
data.head()

,Unnamed: 0,text,class
0,2,ex wife threatening suiciderecently left wife ...,1
1,3,weird get affected compliments coming someone ...,0
2,4,"finally 2020 almost over... never hear ""2020 b...",0
3,8,need helpjust help im crying hard,1
4,9,"i’m losthello, name adam (16) i’ve struggling ...",1


In [57]:
data.isnull().sum()

Unnamed: 0    0
text          0
class         0
dtype: int64

In [55]:
data.to_csv('preprocessed_data')